<h1> Chunking BERT <h1>

# Conda update

In [6]:
conda update conda

done
| lving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::hvplot==0.8.4=py311hecd8cb5_0
  - defaults/osx-64::nltk==3.8.1=py311hecd8cb5_0
  - defaults/osx-64::nbclassic==0.5.5=py311hecd8cb5_0
  - defaults/osx-64::jupyter_server_fileid==0.9.0=py311hecd8cb5_0
  - defaults/osx-64::imbalanced-learn==0.10.1=py311hecd8cb5_1
  - defaults/osx-64::aext-core-server==4.1.0=py311hecd8cb5_0
  - defaults/osx-64::typing-extensions==4.7.1=py311hecd8cb5_0
  - defaults/osx-64::holoviews==1.17.0=py311hecd8cb5_0
  - defaults/osx-64::anaconda-cloud-auth==0.7.2=py311hecd8cb5_0
  - defaults/osx-64::jupyter_server==2.14.1=py311hecd8cb5_0
  - defaults/osx-64::jupyterlab_server==2.27.3=py311hecd8cb5_0
  - defaults/osx-64::aext-core==4.1.0=py311hecd8cb5_jl4_0
  - defaults/noarch::argon2-cffi==21.3.0=pyhd3eb1b0_0
  - defaults/osx-64::anaconda-client==1.13.0=py311hecd8cb5_0
  - defaults/osx-64::ae

# Installing packages

In [3]:
conda create -n scispacy-env python=3.11

doneing environment: / 


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 25.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.3.0



## Package Plan ##

  environment location: /Users/mike/anaconda3/envs/scispacy-env

  added / updated specs:
    - python=3.11


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.0.16             |       h184c1cd_0         4.6 MB
    pip-25.0                   |  py311hecd8cb5_0         2.9 MB
    python-3.11.11             |       h4d6d9e5_0        15.7 MB
    setuptools-75.8.0          |  py311hecd8cb5_0         2.2 MB
    wheel-0.45.1               |  py311hecd8cb5_0         156 KB
    ------------------------------------------------------------
                        

In [1]:
pip install scispacy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz (14.8 MB)
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import numpy as np
import spacy

from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_sci_sm")

/Users/mike/anaconda3/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


# Tokenization and chunking

In [8]:
# folder path
folder_path = "Theses_clean_data/final_patent_text.csv"

In [10]:
df = pd.read_csv(folder_path)

In [12]:
df.head()

,ApplicationNumber,Titles,Abstracts,Claims,IPCs,CPCs,ProbablePatentAssignee,EarliestPriorityDate,claim_length,abstract_length,Year
0,US20220083465,system and method for tissue intervention via ...,one embodiment is directed to a minimally inva...,. a minimally invasive system for treating a ...,"['A61B34/10', 'A61B34/20']","['A61B2034/107', 'A61B2034/2051', 'A61B2034/20...",PETAL SURGICAL INC,2021-12-16,13540,178,2021
1,US20180497284,treatment of a disease of the gastrointestinal...,this disclosure features methods and compositi...,. a method of treating a disease of the gastr...,"['A61K31/7105', 'A61M31/00', 'C12N15/11', 'C12...","['A61B1/04', 'A61B5/06', 'A61B5/062', 'A61B5/0...",BIORA THERAPEUTICS INC,2017-03-30,12792,24,2017
2,US20190467784,treatment of a disease of the gastrointestinal...,this disclosure features methods and compositi...,. a method of treating a disease of the gastr...,"['A61B1/04', 'A61B1/273', 'A61B5/00', 'A61K31/...","['A61B1/041', 'A61B1/273', 'A61B5/0071', 'A61B...",BIORA THERAPEUTICS INC,2016-12-14,12711,17,2016
3,US20170465360,treatment of a disease of the gastrointestinal...,this disclosure features methods and compositi...,. a method of treating a disease of the gastr...,"['A61B5/00', 'A61B5/01', 'A61B5/07', 'A61B5/14...","['A61B1/041', 'A61B2010/0061', 'A61B5/0075', '...",BIORA THERAPEUTICS INC,2016-12-14,12694,17,2016
4,US20210295221,apparatus and methods for use with image-guide...,a procedure is performed with respect to a ske...,. a method for performing a procedure with re...,"['A61B34/10', 'A61B34/30', 'A61B90/00', 'G06T7...","['A61B2034/105', 'A61B2034/107', 'A61B2034/204...",VUZE MEDICAL LTD,2018-11-22,12632,139,2018


In [24]:
sample_df = df[['ApplicationNumber','Claims', 'ProbablePatentAssignee','EarliestPriorityDate']].sample(n=1, random_state=100).dropna()

In [26]:
sample_df

,ApplicationNumber,Claims,ProbablePatentAssignee,EarliestPriorityDate
43776,US20230142093,". an apparatus, comprising: an intravascular ...",PHILIPS KONINKLL NV,2016-11-11


In [18]:
print(sample_df['Claims'].iloc[0])

 . an apparatus, comprising: an intravascular ultrasound (ivus) imaging device comprising: a catheter configured to be positioned within a blood vessel and comprising a proximal portion and a distal portion; an ultrasound transducer array coupled to the distal portion and comprising a rolled configuration, at least one integrated circuit chip coupled to the distal portion and configured to: control the ultrasound transducer array to emit ultrasound energy and receive ultrasound echoes associated with the ultrasound energy; and perform a first processing of ultrasound echo signals representative of the ultrasound echoes; a signal processor coupled to the proximal portion and configured to: receive the ultrasound echo signals after the first processing has been performed; and perform a second processing of ultrasound echo signals; a transceiver coupled to the proximal portion and configured to receive the ultrasound echo signals after the second processing has been performed; and an ante

# Chunking & Tokenization

In [72]:
def chunk_and_tokenize(df, text_column, app_number_column="ApplicationNumber", model="en_core_sci_sm", chunk_size=300):
    nlp = spacy.load(model)

    new_rows = []  

    for index, row in df.iterrows():
        text = row[text_column]
        app_number = row[app_number_column]
        doc = nlp(text)

        # Tokenization & Chunking
        words = [token.text for token in doc if not token.is_punct and not token.is_space]
        num_chunks = (len(words) // chunk_size) + (1 if len(words) % chunk_size > 0 else 0)  # Count chunks

        for chunk_id, i in enumerate(range(0, len(words), chunk_size), start=1):
            chunk = " ".join(words[i:i + chunk_size])
            new_rows.append({
                "original_index": index,
                "ApplicationNumber": app_number,
                "chunk_id": chunk_id,
                "num_chunks": num_chunks,
                "chunk": chunk,
            })

    return pd.DataFrame(new_rows)

In [84]:
test_chunked = chunk_and_tokenize(sample_df, text_column="Claims")

In [86]:
test_chunked.head()

,original_index,ApplicationNumber,chunk_id,num_chunks,chunk
0,43776,US20230142093,1,3,an apparatus comprising an intravascular ultra...
1,43776,US20230142093,2,3,element to emit the ultrasound energy or in a ...
2,43776,US20230142093,3,3,the apparatus of claim further comprising a ha...


In [88]:
chunked_df = chunk_and_tokenize(df, text_column="Claims")

In [89]:
# save df to csv
chunked_df.to_csv('chunked_df.csv', index=False)

In [96]:
print(f"Number of unique Application no's: {len(chunked_df.ApplicationNumber.unique())}")

Number of unique Application no's: 51626


In [104]:
chunked_df.head()

,original_index,ApplicationNumber,chunk_id,num_chunks,chunk
0,0,US20220083465,1,43,a minimally invasive system for treating a tar...
1,0,US20220083465,2,43,load sensor a joint torque sensor a strain gau...
2,0,US20220083465,3,43,system of claim wherein the hifu treatment tra...
3,0,US20220083465,4,43,in part by an acoustic fountain reaction assoc...
4,0,US20220083465,5,43,data the system of claim further comprising a ...


In [106]:
chunked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219715 entries, 0 to 219714
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   original_index     219715 non-null  int64 
 1   ApplicationNumber  219715 non-null  object
 2   chunk_id           219715 non-null  int64 
 3   num_chunks         219715 non-null  int64 
 4   chunk              219715 non-null  object
dtypes: int64(3), object(2)
memory usage: 8.4+ MB
